# Production use
## TensorFlow.js

In [3]:
!ls -laF

total 40
drwxr-xr-x 1 root root 4096 May 27 14:26 ./
drwxr-xr-x 1 root root 4096 May 27 09:32 ../
-rw-r--r-- 1 root root 2547 May 27 09:31 adc.json
drwxr-xr-x 1 root root 4096 May 27 09:31 .config/
drwxr-xr-x 3 root root 4096 May 27 09:32 dataset/
drwxr-xr-x 8 root root 4096 May 27 13:28 logs/
drwxr-xr-x 3 root root 4096 May 27 14:26 models/
drwxr-xr-x 1 root root 4096 May 22 16:22 sample_data/


In [4]:
!pip install tensorflowjs

Convert model

In [5]:
!tensorflowjs_converter --input_format keras \
                        models/model_transfered_mobilenet.h5 \
                        prod/tfjs

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'
ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'


In [6]:
!pip freeze | grep numpy

msgpack-numpy==0.4.3.2
numpy==1.15.1


In [7]:
!pip install numpy==1.16.1

     |████████████████████████████████| 17.3MB 3.4MB/s 
ERROR: thinc 6.12.1 has requirement wrapt<1.11.0,>=1.10.0, but you'll have wrapt 1.11.1 which is incompatible.
ERROR: tensorflowjs 1.1.2 has requirement numpy==1.15.1, but you'll have numpy 1.16.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.11.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.15.1
    Uninstalling numpy-1.15.1:
      Successfully uninstalled numpy-1.15.1


In [0]:
!tensorflowjs_converter --input_format keras \
                        models/model_transfered_mobilenet.h5 \
                        prod/tfjs

In [2]:
!ls -laF prod/tfjs/

total 31216
drwxr-xr-x 2 root root    4096 May 27 14:32 ./
drwxr-xr-x 3 root root    4096 May 27 14:32 ../
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard1of8.bin
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard2of8.bin
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard3of8.bin
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard4of8.bin
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard5of8.bin
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard6of8.bin
-rw-r--r-- 1 root root 4194304 May 27 14:33 group1-shard7of8.bin
-rw-r--r-- 1 root root 2586132 May 27 14:33 group1-shard8of8.bin
-rw-r--r-- 1 root root    4930 May 27 14:33 model.json


## App base

In [3]:
%%writefile prod/tfjs/index.html


<html>
<head>
    <!-- Load the latest version of TensorFlow.js -->
    <script src="https://unpkg.com/@tensorflow/tfjs"></script>
    <script src="https://unpkg.com/@tensorflow-models/mobilenet"></script>
    <script src="https://unpkg.com/@tensorflow-models/knn-classifier"></script>

</head>
<body>
    <style>
        .pred-container {
            margin-bottom: 20px;
        }
        .pred-container > div {
            display: inline-block;
            margin-right: 20px;
            vertical-align: top;
        }
        .row {
            display: table-row;
        }
        .cell {
            display: table-cell;
            padding-right: 20px;
        }
        #file-container {
            margin-bottom: 20px;
        }
        </style>
    <img id="img" crossOrigin src="shoe.png" width=192 height=192/>
    <div id="predictions"></div>


    <!-- Load index.js after the content of the page -->
    <script src="main.js"></script>
</body>
</html>

Writing prod/tfjs/index.html


In [4]:
%%writefile prod/tfjs/main.js



IMAGE_SIZE = 192
CLASSES = ['converse chuck 70', 'Exaggerated Soles', 'nike air max 90 essentials', 'nike react', 'Tie Dye sneakers']

const predictionsElement = document.getElementById('predictions');

async function app() {
  console.log('Loading model..');

  // Load the model.
  const model = await tf.loadLayersModel('/model.json');
  console.log('Sucessfully loaded model');

  // Make a prediction through the model on our image.
  const imgEl = document.getElementById('img');


  const result = await classify(model, imgEl);
  console.log(result);
}


async function classify(model, imgElement) {
  console.log('Predicting...');

  // The first start time includes the time it takes to extract the image
  // from the HTML and preprocess it, in additon to the predict() call.
  const startTime1 = performance.now();
  // The second start time excludes the extraction and preprocessing and
  // includes only the predict() call.
  let startTime2;
  const logits = tf.tidy(() => {
    // tf.browser.fromPixels() returns a Tensor from an image element.
    const img = tf.browser.fromPixels(imgElement, 3).toFloat();

    // Normalize the image from [0, 255] to [-1, 1].
    // const offset = tf.scalar(127.5);
    // const normalized = img.sub(offset).div(offset);
    normalized = img

    // Reshape to a single-element batch so we can pass it to predict.
    const batched = normalized.reshape([1, IMAGE_SIZE, IMAGE_SIZE, 3]);



    startTime2 = performance.now();
    // Make a prediction through mobilenet.
    return model.predict(batched);
  });

  console.log("Logits from model: ", logits)

  // Convert logits to probabilities and class names.
  classes = await getTopKClasses(logits, 2);
  console.log("Predicred classes: ", classes)

  const totalTime1 = performance.now() - startTime1;
  const totalTime2 = performance.now() - startTime2;
  console.log(`Done in ${Math.floor(totalTime1)} ms ` +
      `(not including preprocessing: ${Math.floor(totalTime2)} ms)`);

  showResults(imgElement, classes);
}

async function getTopKClasses(logits, topK) {
  const values = await logits.data();

  const valuesAndIndices = [];
  for (let i = 0; i < values.length; i++) {
    valuesAndIndices.push({value: values[i], index: i});
  }
  valuesAndIndices.sort((a, b) => {
    return b.value - a.value;
  });
  const topkValues = new Float32Array(topK);
  const topkIndices = new Int32Array(topK);
  for (let i = 0; i < topK; i++) {
    topkValues[i] = valuesAndIndices[i].value;
    topkIndices[i] = valuesAndIndices[i].index;
  }

  const topClassesAndProbs = [];
  for (let i = 0; i < topkIndices.length; i++) {
    topClassesAndProbs.push({
      className: CLASSES[topkIndices[i]],
      probability: topkValues[i]
    })
  }
  return topClassesAndProbs;
}

function showResults(imgElement, classes) {
  const predictionContainer = document.createElement('div');
  predictionContainer.className = 'pred-container';

  const imgContainer = document.createElement('div');
  imgContainer.appendChild(imgElement);
  predictionContainer.appendChild(imgContainer);

  const probsContainer = document.createElement('div');
  for (let i = 0; i < classes.length; i++) {
    const row = document.createElement('div');
    row.className = 'row';

    const classElement = document.createElement('div');
    classElement.className = 'cell';
    classElement.innerText = classes[i].className;
    row.appendChild(classElement);

    const probsElement = document.createElement('div');
    probsElement.className = 'cell';
    probsElement.innerText = classes[i].probability.toFixed(3);
    row.appendChild(probsElement);

    probsContainer.appendChild(row);
  }
  predictionContainer.appendChild(probsContainer);

  predictionsElement.insertBefore(
      predictionContainer, predictionsElement.firstChild);
}


app();

Writing prod/tfjs/main.js


### Serve our app directly from Colab



Serve files with python server module like 

```bash
python3 -m http.server
```

To see the result of work of the server we need `ngrock` for opening the port to outside

In [5]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2019-05-27 14:33:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.174.228.92, 34.195.49.195, 52.203.102.189, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.174.228.92|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16648024 (16M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  15.88M  42.3MB/s    in 0.4s    

2019-05-27 14:33:31 (42.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [16648024/16648024]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


Now let's make `ngrok` to proxy the traffic from the localhost

In [0]:
get_ipython().system_raw('./ngrok http 8000 &')

In [21]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://13f9b62d.ngrok.io


In [22]:
!cd prod/tfjs/ && python3 -m http.server

Serving HTTP on 0.0.0.0 port 8000 (http://0.0.0.0:8000/) ...
127.0.0.1 - - [27/May/2019 14:37:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:36] "GET /shoe.png HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:36] "GET /main.js HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:36] "GET /model.json HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:36] "GET /group1-shard1of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:37] "GET /group1-shard4of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:37] "GET /group1-shard3of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:37] "GET /group1-shard2of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:37] "GET /group1-shard5of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:37] "GET /group1-shard6of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:38] "GET /group1-shard7of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:39] "GET /group1-shard8of8.bin HTTP/1.1" 200 -
127.0.0.1 - - [27/May/2019 14:37:3

Now you can open the `ngrok` URL and being able to see predictions

### Links


*  https://www.tensorflow.org/js/tutorials/conversion/import_keras
*  https://github.com/tensorflow/tfjs-examples/blob/master/mobilenet/index.js
* https://js.tensorflow.org/api/0.11.2/

